In [1]:
from snowflake.nodes import *
from snowflake.stencil_compiler import *
from snowflake.analytics import get_shadow, validate_stencil
from snowflake.utils import swap_variables
import numpy as np
import inspect
!ctree -cc

ctree looking for relative cache directories named compiled, checking directories under this one



In [2]:
sizes = (33, 33, 33)
data = np.zeros(sizes)
swap = np.zeros_like(data)

In [3]:
smooth_sc = StencilComponent('data', WeightArray([[[1/4.0, 1/2.0, 1/4.0]]]))
smooth_stencil = Stencil(smooth_sc, 'swap', ((1, -1, 1),(1, -1, 1),(1, -1, 1)))

In [4]:
compiler = CCompiler()
kern = compiler.compile(smooth_stencil)
kern(swap, data)